In [1]:
# First import necessary libraries
import pandas as pd
from sklearn.ensemble import StackingRegressor
# Neural networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add, Input, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Embedding
from tensorflow.keras.layers import Flatten, Concatenate
from tensorflow.keras import regularizers
from keras.regularizers import l1
from keras.regularizers import l2
from tensorflow.keras import regularizers
# Wrapper to make neural network compitable with StackingRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# Linear model as meta-learn
from sklearn.linear_model import LinearRegression
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Create generic dataset for regression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier 

In [2]:
# Create regression dataset
data = pd.read_csv('train2.csv')
data.drop('Id',axis=1 , inplace=True)
tfidf = TfidfVectorizer()
#intializing the TfidVectorizer object... helps vectorize and normalize the data using a logarithmic function 


In [3]:
CommentsToBeTokenized = data['Comment'] 
tfidf_result=tfidf.fit_transform(CommentsToBeTokenized).toarray()
vectorizer = TfidfVectorizer(max_features= 6000)
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)
X_train, X_val, y_train, y_val = train_test_split(tfidf_result, data['Topic'], test_size= 0.2, random_state = 110)

In [4]:
X_train

,00,000,01,019,02,020,021,03,04,07,...,yourself,youtu,youtube,yt,yup,zeolites,zero,zinc,zone,zp
8504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
7383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [5]:
def create_neural_network(input_shape, depth=4, batch_mod=2, num_neurons=20, drop_rate=0.1, learn_rate=.01,
                      r1_weight=0.02,
                      r2_weight=0.02):
    '''A neural network architecture built using keras functional API'''
    act_reg = l1(r2_weight)
    kern_reg = l1(r1_weight)
    
    inputs = Input(shape=(input_shape,))
    batch1 = BatchNormalization()(inputs)
    hidden1 = Dense(num_neurons, activation='relu', kernel_regularizer=kern_reg, activity_regularizer=act_reg)(batch1)
    dropout1 = Dropout(drop_rate)(hidden1)
    hidden2 = Dense(int(num_neurons/2), activation='relu', kernel_regularizer=kern_reg, activity_regularizer=act_reg)(dropout1)
    
    skip_list = [batch1]
    last_layer_in_loop = hidden2
    
    for i in range(depth):
        added_layer = concatenate(skip_list + [last_layer_in_loop])
        skip_list.append(added_layer)
        b1 = None
        #Apply batch only on every i % N layers
        if i % batch_mod == 2:
            b1 = BatchNormalization()(added_layer)
        else:
            b1 = added_layer
        
        h1 = Dense(num_neurons, activation='relu', kernel_regularizer=kern_reg, activity_regularizer=act_reg)(b1)
        d1 = Dropout(drop_rate)(h1)
        h2 = Dense(int(num_neurons/2), activation='relu', kernel_regularizer=kern_reg, activity_regularizer=act_reg)(d1)
        d2 = Dropout(drop_rate)(h2)
        h3 =  Dense(int(num_neurons/2), activation='relu', kernel_regularizer=kern_reg, activity_regularizer=act_reg)(d2)
        d3 = Dropout(drop_rate)(h3)
        h4 =  Dense(int(num_neurons/2), activation='relu', kernel_regularizer=kern_reg, activity_regularizer=act_reg)(d3)
        last_layer_in_loop = h4
        c1 = concatenate(skip_list + [last_layer_in_loop])
        output = Dense(1, activation='sigmoid')(c1)
    
    model = Model(inputs=inputs, outputs=output)
    optimizer = Adam()
    optimizer.learning_rate = learn_rate
    
    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['accuracy'])
    return model

In [6]:
#This code is for regression... have to figure out how to mold this into a classifier thing 

def get_stacking(input_shape=None):
    '''A stacking model that consists of CatBoostRegressor,
    XGBRegressor, a linear model, and some neural networks'''
    # First we create a list called "level0", which consists of our base models"
    # These models will get passed down to the meta-learner later
    level0 = list()
    level0.append(('KNeighborsClassifier',  KNeighborsClassifier(n_neighbors= 4)))
    level0.append(('MultinomialNB', MultinomialNB(alpha = 0.1)))
    level0.append(('RandomForest', RandomForestClassifier(max_depth = 9)))
#Create 5 neural networks using our function above
    for i in range(5):
        # Wrap our neural network in a Keras Regressor to make it
        #compatible with StackingRegressor
        keras_class = KerasClassifier(
                create_neural_network, # Pass in function
                input_shape=input_shape, # Pass in the dimensions to above function
                epochs=6,
                batch_size=32,
                verbose=0)
        keras_class._estimator_type = "classifier"
        # Append to our list
        level0.append(('nn_{num}'.format(num=i), keras_class))
# The "meta-learner" designated as the level1 model
    # In my experience Linear Regression performs best
    # but feel free to experiment with other models
    level1 = MultinomialNB() 
# Create the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=2, verbose=1)
    return model

In [7]:
#Get our input dimensions for neural network
input_dimensions = len(X_train.columns)
# Create stacking model
model = get_stacking(input_dimensions)
model.fit(X_train, y_train.values.ravel())
# Creating a temporary dataframe so we can see how each of our models performed
temp = pd.DataFrame(y_val)
# The stacked models predictions, which should perform the best
temp['stacking_prediction'] = model.predict(X_val)
# Get each model in the stacked model to see how they individually perform
for m in model.named_estimators_:
        temp[m] = model.named_estimators_[m].predict(X_val)
# See how each of our models correlate with our target
print(temp.corr()['target'])
# See what our meta-learner is thinking (the linear regression)
for coef in zip(model.named_estimators_, model.final_estimator_.coef_):
    print(coef)

C:\Users\serap\AppData\Local\Temp\ipykernel_4708\4277335599.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_class = KerasClassifier(


ValueError: The estimator KNeighborsClassifier should be a regressor.

In [8]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
import numpy as np
import warnings

In [14]:
clf1 = KNeighborsClassifier(n_neighbors= 4, weights = 'distance', leaf_size = 20 )
clf2 = RandomForestClassifier(max_depth = 9)
clf3 = MultinomialNB(alpha = 0.1)
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
                          use_probas=True,
                          average_probas=False,
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([clf1, clf2, clf3, sclf], 
                      ['KNN', 
                       'Random Forest', 
                       'Multinomial Naive Bayes',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X_train, y_train, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))
#Other cross validation methods had a run time error... time to try Keras! 

3-fold cross validation:

Accuracy: 0.42 (+/- 0.01) [KNN]
Accuracy: 0.48 (+/- 0.00) [Random Forest]
Accuracy: 0.71 (+/- 0.00) [Multinomial Naive Bayes]
Accuracy: 0.50 (+/- 0.01) [StackingClassifier]


In [15]:
from sklearn.ensemble import StackingClassifier as newSC 
base_learners = [
                 ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42, max_depth = 9)),
                 ('rf_2', KNeighborsClassifier(n_neighbors=5)),
                 ('rf_3', MultinomialNB(alpha = 0.1))
                ]

# Initialize Stacking Classifier with the Meta Learner
clf = newSC(estimators=base_learners, final_estimator=LogisticRegression())
#

In [16]:
clf.fit(X_train, y_train)
#Taking way longer than expected.... 

StackingClassifier(estimators=[('rf_1',
                                RandomForestClassifier(max_depth=9,
                                                       n_estimators=10,
                                                       random_state=42)),
                               ('rf_2', KNeighborsClassifier()),
                               ('rf_3', MultinomialNB(alpha=0.1))],
                   final_estimator=LogisticRegression())

In [17]:
ypred1 = clf.predict(X_train)
ypred = clf.predict(X_val)
ytrain = y_train[0:1739]
ytest = y_val[0:1739]
ypred1 = ypred1[0:1739]
ypred = ypred[0:1739]
print("Training Results:\n")
print(classification_report(ytrain, ypred1))
print("\nTesting Results:\n")
print(classification_report(ytest, ypred))

Training Results:

              precision    recall  f1-score   support

     Biology       0.82      0.92      0.87       745
   Chemistry       0.86      0.82      0.84       568
     Physics       0.90      0.79      0.84       426

    accuracy                           0.85      1739
   macro avg       0.86      0.84      0.85      1739
weighted avg       0.85      0.85      0.85      1739


Testing Results:

              precision    recall  f1-score   support

     Biology       0.74      0.82      0.78       791
   Chemistry       0.67      0.63      0.65       550
     Physics       0.76      0.64      0.70       398

    accuracy                           0.72      1739
   macro avg       0.72      0.70      0.71      1739
weighted avg       0.72      0.72      0.72      1739



In [23]:
import joblib

In [25]:
filename = 'finalized_model.sav'
joblib.dump(clf, filename)
#300 Megabytes... cannot use 

['finalized_model.sav']

In [26]:
CommentsToBeTokenized = data['Comment'] 
tfidf_result=tfidf.fit_transform(CommentsToBeTokenized).toarray()
vectorizer = TfidfVectorizer(max_features= 6000)
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)
X_train, X_val, y_train, y_val = train_test_split(tfidf_result, data['Topic'], test_size= 0.2, random_state = 110)

In [27]:
clf3 = MultinomialNB(alpha = 0.1)

In [29]:
clf3.fit(X_train, y_train)

MultinomialNB(alpha=0.1)

In [30]:
filename = 'finalized_model.sav'
joblib.dump(clf3, filename)

['finalized_model.sav']